### IMPORTS

In [1]:
import torch
import torchvision
import numpy as np
import time
import ipywidgets
import cv2
from IPython.display import display
from jetracer.nvidia_racecar import NvidiaRacecar
from jetcam.csi_camera import CSICamera
from torch2trt import torch2trt

from autopilot_utils import preprocess_image
from autopilot_model import AutopilotModel

<br>

### HYPERPARAMETERS

In [2]:
MODELS_DIR = "/home/greg/models/"
NAME = "camera-basic_motion_0_resnet18_1"
MODEL_PATH = MODELS_DIR + NAME + ".pth"

STEERING_OFFSET = 0.05
THROTTLE_GAIN = 0.6

WIDTH = 224
HEIGHT = 224
CHANNELS = 3

SHOW_CAMERA_PREVIEW = False

<br>

### MODEL

In [3]:
model = AutopilotModel(pretrained=False)
model.load_from_path(MODEL_PATH)
model.eval()

x = torch.ones((1, CHANNELS, HEIGHT, WIDTH)).cuda()
model_trt = torch2trt(model, [x], fp16_mode=True)

<br>

### CAR

In [4]:
car = NvidiaRacecar()
car.throttle_gain = THROTTLE_GAIN
car.steering_offset = STEERING_OFFSET

<br>

### CAMERA

In [5]:
camera = CSICamera(width=WIDTH, height=HEIGHT)

In [6]:
if SHOW_CAMERA_PREVIEW:
    image_widget = ipywidgets.Image(format='jpeg')
    display(image_widget)

<br>

### CONTROL LOOP

In [11]:
while True:
    start_time = time.time()
    
    image = camera.read()
    
    if SHOW_CAMERA_PREVIEW:
        encoded_image = cv2.imencode('.jpg', image)[1]
        image_widget.value = bytes(encoded_image)
        
    image = preprocess(image)
    output = model_trt(image).detach().clamp(-1, 1).cpu().numpy().flatten()
    steering = float(output[0])
    throttle = float(output[1])
    car.steering = steering
    car.throttle = throttle
    
    fps = 1 / (time.time() - start_time)
    print("fps: " + str(fps) + ", steering: " + str(steering) + ", throttle: " + str(throttle), end="\r")

KeyboardInterrupt: 

In [12]:
car.throttle_gain = 0.6
car.throttle = 0
car.steering = 0